# 第8回 その3: 混合正規分布
5教科テストデータに対して 混合正規分布を用いたクラス分類を行ってみましょう。


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/08"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/08/
# 現在のフォルダの中身を表示
%ls

`gokyouka.csv`というデータが表示されていることを確認してください。

## ステップ1: データの読み込み
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`gokyouka.csv` を読み込みます。  
このデータは第2回のレポート課題で使用した，5教科のテストの点数を示したデータ(report02_input.csv)です。

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('gokyouka.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# numpy用データ(ndarray型) に変換する。
X = csv_data.to_numpy()

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

## ステップ2: 主成分分析によるデータの2次元への圧縮  
08_02_kmeans.ipynbと同様に，主成分分析を使って2次元に圧縮します。

In [ ]:
# データの標準化
X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# 分散共分散行列の計算
cov = np.cov(X_norm.T, bias=True)
# 固有値分解
eig_val, eig_vec =np.linalg.eig(cov)

# 固有値を大きい順に並び替える
order = np.argsort(eig_val)[::-1]
eig_val = eig_val[order]
eig_vec_pca = eig_vec[:,order]

# 第二主成分までの固有ベクトルを用いて次元圧縮
X_pca = np.dot(X_norm, eig_vec_pca[:,:2])

# 二次元データのプロット
plt.figure(figsize=(10,6))
plt.scatter(X_pca[:,0], X_pca[:,1], color='k')
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.show()

主成分分析によって2次元圧縮したデータが `X_pca` に格納されました。  
以降は `X_pca` に対して k-means を行います。  

## ステップ3: K-means による3クラスのクラスタリング  
比較のため，まずはK-meansによるクラスタリングを行います。  
以下は k-means の関数の実装(06_02_kmeans.ipynb参照)です。

In [ ]:
def euclidean_distance(x, y):
  '''
      ユークリッド距離を計算して出力する
      x, y: 距離を測りたい2個の多次元ベクトル
  '''
  dist = np.sum((x-y)**2)
  return dist

def k_means(centroid, data):
  '''
     k-means によるセントロイドの更新を1回行う。
     入力
     centroid: 現在のセントロイド
     data: データ
     出力
     new_centroid: 更新後のセントロイド
     assigned_class: 各データが振り分けられたクラス番号
  '''
  # クラス数を取得
  num_classes, num_dimensions = np.shape(centroid)
  # データのサンプル数と次元数を取得
  num_samples, num_dimensions = np.shape(data)

  # 各データに対して割り当てるクラス番号
  assigned_class = np.zeros(num_samples)

  # 各データに対して，最もセントロイドが近いクラスを割り当てる
  for n in range(num_samples):
    # 各クラスのセントロイドとの距離を測る
    dists = np.zeros(num_classes)
    for k in range(num_classes):
      # n番目のデータとk番目のセントロイドの距離
      dists[k] = euclidean_distance(data[n,:], centroid[k,:])

    # セントロイドとの距離が最小のクラス番号を割り当てる
    assigned_class[n] = np.argmin(dists)
  
  # セントロイドの更新
  new_centroid = np.zeros((num_classes, num_dimensions))
  for k in range(num_classes):
    # k番目のクラスに該当するデータ data[assigned_class==k] の平均値を
    # k番目のクラスの新たなセントロイドとする
    new_centroid[k] = np.mean(data[assigned_class==k,:], axis=0)
  
  return new_centroid, assigned_class

分割するクラスの数 $K=3$ としてセントロイド初期値を設定し，k-meansを実行します。

In [ ]:
#クラス数
K = 3

#
# セントロイド初期値の設定(06_01_kmeans.ipynb参照)
#
# 乱数シードの設定(ここではシードを5としていますが，何でも良いです)
np.random.seed(5)
# 0～99 をランダムに並び替える
shuffle_index = np.random.permutation(np.arange(num_samples))
# 並び替えた数の先頭 K個を取り出して、セントロイドの初期値とする。
centroid_index = shuffle_index[:K]
initial_centroid = X_pca[centroid_index,:]


#
# K-means を実行する(06_01_kmeans.ipynb参照)
#

# 最大更新回数を100とする
max_iter = 100

centroid = initial_centroid

# ループの回数は max_iter とする
for n in range(max_iter):
  print('Iteration: ' + str(n+1))

  # k-meansによるセントロイドの更新を実施
  new_centroid, assigned_class = k_means(centroid, X_pca)

  # セントロイドが変わっていない，
  # つまり更新前後のセントロイドの移動距離が0の場合，ループを抜ける
  if np.sum((new_centroid - centroid)**2) == 0:
    print('centroids converged')
    break
  
  # セントロイドの更新
  centroid = new_centroid

結果をプロットしてみましょう。

In [ ]:
colors = ['r', 'b', 'g', 'c', 'm', 'y']
# 二次元データのプロット
plt.figure(figsize=(10,6))
# セントロイドを"x"印でプロット 
for k in range(K):
  c = colors[k%6] # プロットの色を，クラス番号によって自動的に変える。
  # 全データのプロット
  plt.scatter(X_pca[assigned_class==k,0], X_pca[assigned_class==k,1], color=c, label='class: '+str(k+1))
  # セントロイドを "x"印でプロット
  plt.scatter(centroid[k,0], centroid[k,1], label='centroid: '+str(k+1), color=c, marker='x', s=100)
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.legend()
plt.show()

上記の結果では，成績が普通のクラス(文系)と成績が悪いクラスが緑色のクラス3でまとまってクラス化されていることから，この結果は直感と異なるクラスタリングがされていると言えます。


## ステップ4: 混合正規分布によるクラスタリング  
では混合正規分布を使って，3クラスのクラスタリングを実行してみましょう。  

ここでは機械学習系のライブラリであるscikit-learn (sklearn)からmixtureというモジュールをインポートすることで，混合正規分布を実行しています。  
（ソースコードの実装はレポート課題です。）

In [ ]:
from sklearn import mixture

gmm = mixture.GaussianMixture(n_components=K)
gmm_res = gmm.fit(X_pca)
# クラスタリング結果
assigned_class_gmm = gmm_res.predict(X_pca)
# 平均値ベクトル
means = gmm_res.means_

結果をプロットしてみましょう。

In [ ]:
colors = ['r', 'b', 'g', 'c', 'm', 'y']
# 二次元データのプロット
plt.figure(figsize=(10,6))
# セントロイドを"x"印でプロット 
for k in range(K):
  c = colors[k%6] # プロットの色を，クラス番号によって自動的に変える。
  # 全データのプロット
  plt.scatter(X_pca[assigned_class_gmm==k,0], X_pca[assigned_class_gmm==k,1], color=c, label='class: '+str(k+1))
  # セントロイドを "x"印でプロット
  plt.scatter(means[k,0], means[k,1], label='means: '+str(k+1), color=c, marker='x', s=100)
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.legend()
plt.show()

混合正規分布によるクラスタリング結果は，成績が良いクラス，成績が普通のクラス，成績が悪いクラスに分かれていることから，直感に近いクラスタリング結果になっていると言えます。